In [ ]:
import pandas as pd 
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [3]:
data_minhashing=pd.read_csv("/Users/sararedaelli/Desktop/AMAZON REVIEWS/minhashing_results.csv")
df=data_minhashing

In [ ]:

# --- 1. Prepare data ---
# Assuming df['signature'] contains lists or arrays of length 100
X = np.array(df['signature'].tolist())
y = df['score'].values  # true labels

# --- 2. Run K-Means ---
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X)




NameError: name 'np' is not defined

In [ ]:
# --- 3. Evaluate ---
ari = adjusted_rand_score(y, clusters)
nmi = normalized_mutual_info_score(y, clusters)

print(f"Adjusted Rand Index (ARI): {ari:.4f}")
print(f"Normalized Mutual Info (NMI): {nmi:.4f}")

# --- 4. Optional: visualize clusters ---
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(8,6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='tab10', alpha=0.7)
plt.title("K-Means clustering of MinHash signatures")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.colorbar(label="Cluster ID")
plt.show()